<center>
<img src="logo.png" height="900"> 
</center>


#  Анализируем чеки

В этом задании мы будем работать с покупками и чеками. Смотреть за корреляциями в покупках довольно полезно.

> В 1992 году группа по консалтингу в области ритейла компании Teradata под руководством Томаса Блишока провела исследование 1.2 миллиона транзакций в 25 магазинах для ритейлера Osco Drug (Drug Store — формат разнокалиберных магазинов у дома). После анализа всех этих транзакций самым сильным правилом получилось «Между 17:00 и 19:00 чаще всего пиво и подгузники покупают вместе». К сожалению, такое правило показалось руководству Osco Drug настолько контринтуитивным, что ставить подгузники на полках рядом с пивом они не стали. Хотя объяснение паре пиво-подгузники вполне себе нашлось: когда оба члена молодой семьи возвращались с работы домой (как раз часам к 5 вечера), жены обычно отправляли мужей за подгузниками в ближайший магазин. И мужья, не долго думая, совмещали приятное с полезным — покупали подгузники по заданию жены и пиво для собственного вечернего времяпрепровождения.

Для работы будем использовать датасет о продуктовых корзинах: https://www.kaggle.com/heeraldedhia/groceries-dataset

In [1]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')  # стиль для графиков
%matplotlib inline

Подружаем данные и смотрим как они выглядят.

In [2]:
df = pd.read_csv('groceries.csv', sep=',')
df.columns = ['id', 'fielddate', 'product']
print(df.shape)
df.head()

(38765, 3)


,id,fielddate,product
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


## 1. Корреляции

Для начала поработаем с корреляциями в данных. 

__а)__ Какой товар покупался чаще всего? Сохраните название этого товара в переменную `product_name`.

In [6]:
df['product'].value_counts().sort_values(ascending=False)

whole milk               2502
other vegetables         1898
rolls/buns               1716
soda                     1514
yogurt                   1334
                         ... 
frozen chicken              5
bags                        4
baby cosmetics              3
preservation products       1
kitchen utensil             1
Name: product, Length: 167, dtype: int64

In [7]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

product_name = 'whole milk'

# your code here


In [8]:
# проверка, что задание решено корректно
assert len(product_name) == 10

# Аналогичные тесты скрыты от вас

__б)__ Сколько всего уникальных заказов было сделано? Сохраните число заказов в переменную `n_cnt`.

In [9]:
df['id'].nunique()

3898

In [10]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

n_cnt = df['id'].nunique()

# your code here


In [11]:
# проверка, что задание решено корректно
assert n_cnt > 3800
assert n_cnt < 4000

# Аналогичные тесты скрыты от вас

В таблице выше в каждой строке записана информация о покупке конкретного товара. Давайте сделаем табличку размера "число товаров" на "число покупок", чтобы понимать какие товары покупались вместе, а какие нет. 

> Обратите внимание, то здесь задание немного упрощено. Вообще говоря, нам нужно делать агрегацию по паре `fielddate, id`, если мы хотим изучать чеки по-честному. Но мы делаем её только по `id` для того, чтобы не усложнять задание. В качестве необязательного дополнения вы можете после сдачи задания переделать код так, чтобы дата тоже учитывалась при расчётах. 

In [12]:
sparse_sales = pd.pivot_table(df, 
               values='fielddate', 
               index='id', 
               columns='product', 
               fill_value=0, aggfunc='count')

sparse_sales.head()

product,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
id,,,,,,,,,,,,,,,,,,,,,
1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,1,0
1001,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,2,0,0
1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0


В нашей матрице огромное число нулей. Обычно такие матрицы называют разряжеными. Мы занимаем нулями кучу свободной памяти, которую мы могли бы не занимать, если бы хранили данные [в ином виде.](https://cmdlinetips.com/2018/03/sparse-matrices-in-python-with-scipy/)

__в)__ Постройте матрицу корреляций Пирсона. Для этого используйте метод таблицы `.corr`.

In [13]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

sales_correlation = sparse_sales.corr()

# your code here
sales_correlation

product,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
product,,,,,,,,,,,,,,,,,,,,,
Instant food products,1.000000,-0.006936,-0.009420,-0.010825,-0.003470,-0.004007,-0.010419,-0.008275,0.005847,-0.014528,...,0.025355,0.021851,0.005348,0.014025,-0.005670,0.016640,0.002853,0.015981,0.005071,0.018221
UHT-milk,-0.006936,1.000000,0.013806,0.006105,-0.007877,0.018349,0.009462,-0.018785,0.002897,0.020801,...,-0.003803,-0.024466,0.009144,0.006618,-0.012871,-0.011550,0.009990,0.028747,0.006505,0.028753
abrasive cleaner,-0.009420,0.013806,1.000000,-0.006523,-0.002091,-0.002415,-0.013429,-0.004986,0.018970,0.001813,...,0.012922,-0.008507,-0.007390,0.002163,-0.003417,0.010777,0.000107,0.003558,0.008360,-0.009420
artif. sweetener,-0.010825,0.006105,-0.006523,1.000000,0.105251,-0.002775,-0.015432,-0.005730,-0.006645,0.016042,...,0.049392,-0.009776,0.010179,-0.006614,-0.003926,-0.006806,-0.018394,0.029591,-0.024397,-0.010825
baby cosmetics,-0.003470,-0.007877,-0.002091,0.105251,1.000000,-0.000889,-0.004947,-0.001837,0.014798,-0.007983,...,-0.003919,-0.003134,-0.007413,-0.011288,-0.001259,-0.008476,-0.005896,-0.021056,-0.000410,-0.003470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
white bread,0.016640,-0.011550,0.010777,-0.006806,-0.008476,-0.009789,-0.011195,0.005394,0.015850,0.030758,...,0.014921,0.001920,-0.000010,0.069757,0.004792,1.000000,0.046272,0.053497,0.030962,0.016640
white wine,0.002853,0.009990,0.000107,-0.018394,-0.005896,-0.006809,-0.010370,0.022577,0.015060,0.021670,...,-0.005083,0.028101,0.015072,0.000317,0.017039,0.046272,1.000000,0.032862,0.039600,-0.006953
whole milk,0.015981,0.028747,0.003558,0.029591,-0.021056,-0.024316,0.020192,0.032618,0.073856,0.028186,...,0.013874,-0.007158,0.069096,0.060338,0.039298,0.053497,0.032862,1.000000,0.103533,0.043075


Какие продукты сильнее всего коррелируют с яйцами, `domestic eggs` (их чаще всего покупают вместе)?  Сохраните название самого скоррелированного продукта в переменную `top_1`.

In [29]:
sales_correlation['domestic eggs'].sort_values(ascending=False)

product
domestic eggs        1.000000
meat spreads         0.083607
whole milk           0.072512
newspapers           0.064502
rubbing alcohol      0.060533
                       ...   
cream               -0.020814
dessert             -0.021004
cooking chocolate   -0.023280
ready soups         -0.023280
pet care            -0.024206
Name: domestic eggs, Length: 167, dtype: float64

In [16]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

top_1 = 'meat spreads'

# your code here


Какие продукты "мешают" купить яйца, то есть отрицательно коррелируют с их покупкой? Сохраните название продукта с самой большой отрицательной корреляцией в переменную `bottom_1`.

In [17]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

bottom_1 = 'pet care'

# your code here


In [18]:
# проверка, что задание решено корректно
assert len(bottom_1) == 8
assert len(top_1) == 12

# Аналогичные тесты скрыты от вас

Напишите код, который выводит самые коррелируемые товары для случайного продукта из списка `unique_products`.

In [173]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

unique_products = df['product'].unique()

# your code here
def show_most_corr(product):
    corrs = sales_correlation[product].apply(abs).sort_values(ascending=False)
    
    return corrs.index[1], corrs[1]

In [174]:
show_most_corr('domestic eggs')

('meat spreads', 0.08360729411455167)

__г)__ Какие два продукта коррелируют сильнее всего? Положите их название в лист `answer`

In [175]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

best_score = 0
name = None
prod_name = None
for prod in unique_products:
    name_, score = show_most_corr(prod)
    if score > best_score:
        name = name_
        prod_name = prod
        best_score = score
answer = [prod_name, name] 

# your code here
prod_name, name, best_score

('soups', 'preservation products', 0.14346453108373952)

In [176]:
# проверка, что задание решено корректно
assert 'soups' in answer

# Аналогичные тесты скрыты от вас

Конечно же, корреляция — это [не единственный способ искать](https://habr.com/ru/company/ods/blog/353502/) между покупками ассоциативные правила.

## 2. Зависимость. 

В лекции мы с вами сказали, что события $A$ и $B$ называются независимыми, если $P(AB) = P(A)\cdot P(B)$. Отталкиваясь от этого определения, можно ввести другую характеристику, которая показывает, насколько продукты зависят друг от друга, а именно __поддержку (lift).__ 

$$
lift = \frac{P(AB)}{P(A)\cdot P(B)}
$$

Эта метрика описывает отношение зависимости товаров к их независимости. Если оказалось, что `lift = 1`, это означает, что покупка товара $A$ не зависит от покупки товара $B$. Если `lift > 1`, то это означает, что вероятность встретить оба товара в чеке, $P(AB)$ высокая, то есть товары покупают вместе. Если `lift < 1`, это означает, что товары, наоборот, очень часто покупают по-отдельности. 

__д)__ Посчитайте значение нашей метрики для яиц и молока (`'whole milk', 'domestic eggs'`). Запишите получившиеся значение метрики в переменную `answer`.

In [154]:
subset = sparse_sales[
    ((sparse_sales['whole milk'] != 0) | (sparse_sales['domestic eggs'] != 0))
][['whole milk', 'domestic eggs']]
subset.head()

product,whole milk,domestic eggs
id,,
1000,2,0
1001,2,0
1002,1,0
1004,3,0
1006,2,0


In [177]:
p_ab = len(subset[(subset['whole milk']!=0) & (subset['domestic eggs']!=0)]) / len(subset)
p_ab

0.13490891186607581

In [178]:
p_a = (sparse_sales['whole milk'] != 0).sum(axis=0) / sparse_sales.shape[0]
p_a

0.4581836839404823

In [179]:
p_b = (sparse_sales['domestic eggs'] != 0).sum(axis=0) / sparse_sales.shape[0]
p_b

0.1331452026680349

In [180]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

answer = p_ab / (p_a*p_b)

# your code here
answer

2.2114417100824326

In [181]:
def lift(product1, product2):
    subset = sparse_sales[
        ((sparse_sales[product1] != 0) | (sparse_sales[product2] != 0))
    ][[product1, product2]]
    p_ab = len(subset[(subset[product1]!=0) & (subset[product2]!=0)]) / len(subset)
    p_a = (sparse_sales[product1] != 0).sum(axis=0) / sparse_sales.shape[0]
    p_b = (sparse_sales[product2] != 0).sum(axis=0) / sparse_sales.shape[0]
    return p_ab / (p_a * p_b + 1e-10)

In [182]:
# проверка, что задание решено корректно
assert answer < 3
assert answer > 1

# Аналогичные тесты скрыты от вас

__е)__ Посчитайте значение метрики для всех пар продуктов из датасета. Сохраните значения в словарик `dict`. В качестве ключа используете кортеж из пары продуктов. Чтобы удобнее было перебрать все сочетания, используйте `combinations` из модуля `itertools`.

Чтобы при подсчётах не возникало деления на ноль, добавьте к знаменателю маленькое число, например `1e-10`.

In [183]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here


In [184]:
import itertools as it
comb = list(it.combinations(unique_products, 2))

In [185]:
food_lift = {}
for com in comb:
    food_lift[com] = lift(*com)

In [186]:
test = []
for i in range(len(comb)):
    t = len(sparse_sales[(sparse_sales[comb[i][0]]==True)&(sparse_sales[comb[i][1]]==True)])
    if (t == 0) and food_lift[comb[i]] > 0:
        print(comb[i])
        break
    if t != 0:
        test.append(comb[i])

('whole milk', 'cream')


In [187]:
comb[i]

('whole milk', 'cream')

In [188]:
sparse_sales[(sparse_sales[comb[i][0]]==True)&(sparse_sales[comb[i][1]]==True)]

product,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
id,,,,,,,,,,,,,,,,,,,,,


In [189]:
food_lift[('whole milk', 'cream')]

1.1848891972209963

In [190]:
(sparse_sales [['whole milk', 'cream']] >= 1).all(axis=1).sum()

3

In [191]:
ab = (sparse_sales [['whole milk', 'cream']] >= 1).all(axis=1).mean()
a = (sparse_sales ['whole milk'] >= 1).mean()
b = (sparse_sales ['cream'] >= 1).mean()
answer = ab/(a*b)
answer

0.5456326987681971

In [192]:
len(set(test))

303

Сколько пар продуктов покупали вместе хотя бы раз? Запишите ответ в переменную `answer`.

In [193]:
sorted(food_lift.items(), key=lambda x: x[1], reverse=False)

[(('tropical fruit', 'hair spray'), 0.0),
 (('tropical fruit', 'baby cosmetics'), 0.0),
 (('tropical fruit', 'kitchen utensil'), 0.0),
 (('tropical fruit', 'make up remover'), 0.0),
 (('tropical fruit', 'preservation products'), 0.0),
 (('whole milk', 'baby cosmetics'), 0.0),
 (('whole milk', 'bags'), 0.0),
 (('whole milk', 'make up remover'), 0.0),
 (('whole milk', 'preservation products'), 0.0),
 (('pip fruit', 'liqueur'), 0.0),
 (('pip fruit', 'ready soups'), 0.0),
 (('pip fruit', 'make up remover'), 0.0),
 (('pip fruit', 'toilet cleaner'), 0.0),
 (('pip fruit', 'preservation products'), 0.0),
 (('other vegetables', 'kitchen utensil'), 0.0),
 (('rolls/buns', 'preservation products'), 0.0),
 (('pot plants', 'sparkling wine'), 0.0),
 (('pot plants', 'hair spray'), 0.0),
 (('pot plants', 'frozen chicken'), 0.0),
 (('pot plants', 'nuts/prunes'), 0.0),
 (('pot plants', 'potato products'), 0.0),
 (('pot plants', 'artif. sweetener'), 0.0),
 (('pot plants', 'liver loaf'), 0.0),
 (('pot plan

In [194]:
n_n0 = 0
for com in comb:
    if food_lift[com] != 0:
        n_n0 += 1
n_n0

9824

In [195]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

answer = n_n0

# your code here
answer

9824

Для какой пары продуктов метрика $lift$ оказалась самой большой? 

In [196]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here


Сколько раз эти продукты встретились в выборке? Как думаете адеватно ли делать выводы по такому объёму данных? 

In [197]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here


Для какой пары продуктов метрика оказывается самой маленькой? 

In [198]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here


In [199]:
# проверка, что задание решено корректно

assert answer < 10000
assert answer > 9000

# Аналогичные тесты скрыты от вас

## 3. Неоцениваемые задания

Выше мы увидели, что некоторые продукты встречаются в выборке очень редко. Понятное дело, что по ним у нас не получится построить хорошее ассоциативное правило. Попробуйте повторить расчёт той же метрики, но с условием что продукт покупали больше 10 раз. Изучите самые покупаемые вместе продукты и самые непокупаемые вместе продукты. Насколько сильно список отличается от полученного в предыдущем задании? 

In [150]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here


Иногда в чеках пытаются искать __продукты-якоря.__ То есть продукты, которые являются основными. Например: айфон - основной продукт, наушники и чехол - дополнения к нему. Подумайте как можно попытаться найти такие продукты на основе простых метрик, основанных на подсчёте условных вероятностей.

<center>
<img src="https://pp.userapi.com/c638028/v638028181/52e5e/1X-dkzNN1hk.jpg" width="400"> 
</center>